In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc=SparkContext()
spark = SparkSession(sc)

24/01/28 01:12:42 WARN Utils: Your hostname, Tech-Buddy resolves to a loopback address: 127.0.1.1; using 172.17.35.137 instead (on interface eth0)
24/01/28 01:12:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/28 01:12:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/28 01:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/28 01:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/01/28 01:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/01/28 01:12:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
source_df=spark.read.parquet("/home/basu/data/lbd_pq_10k.parquet")
source_df.printSchema()

root
 |-- dir0: string (nullable = true)
 |-- dir1: string (nullable = true)
 |-- dir2: string (nullable = true)
 |-- dir3: string (nullable = true)
 |-- dir4: string (nullable = true)
 |-- measurement_date: date (nullable = true)
 |-- loan_id: integer (nullable = true)
 |-- loan_uuid: string (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- customer_application_id: integer (nullable = true)
 |-- servicing_account_uuid: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- loan_type: string (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- status: string (nullable = true)
 |-- status_prev: string (nullable = true)
 |-- origination_date: date (nullable = true)
 |-- final_take_payment_date: date (nullable = true)
 |-- next_payment_due_date: date (nullable = true)
 |-- past_due_total_to_date: float (nullable = true)
 |-- late_date: date (nullable = true)
 |-- late_date_prev: date (nullable = true)
 |-- days_late_excl_charge_off: integer 

In [4]:
import pandas as pd

In [5]:
#create two another dataframe of metadata of current source data
#and compare those using outer join
# pd_df1=pd.DataFrame(source_std_data.dtypes,columns=['column','data_type'])
pd_df2=pd.DataFrame(source_df.dtypes,columns=['column','data_type'])

In [6]:
old_names=pd_df2['column'].tolist()

In [ ]:
# old_names

In [9]:
lbd_df=pd.read_excel("/home/basu/data/LBD,LBBD,Loan.xlsx",sheet_name='loan_by_day')

In [10]:
#convert pandas df into spark df
lbd_spark=spark.createDataFrame(lbd_df)

In [ ]:
#select 'Target Field Name NEW(Display Name)','Source Field Names' where 'Processing Order' ==1 and 'Pre Processing Rules' == 'rename'

In [11]:
rename_df=lbd_spark.select(lbd_spark['Target Field Name NEW(Display Name)'],lbd_spark['Source Field Names']).filter((lbd_spark['Processing Order']==1)&(lbd_spark['Pre Processing Rules']=='rename'))

In [12]:
rename=rename_df.toPandas()

In [13]:
rename

,Target Field Name NEW(Display Name),Source Field Names
0,loan_customer_application_id,customer_application_id
1,prev_ops_loan_status,status_prev
2,ops_loan_status,loan_status
3,past_due_total_amount_to_date,past_due_total_to_date
4,prev_late_date,late_date_prev
...,...,...
204,total_paid_tco_recoveries_amount,total_paid_tco_recovery
205,total_refund_tco_recoveries_amount,total_refunded_tco_recovery
206,total_refinance_payoff_tco_recoveries_amount,total_refinance_payoffs_tco_recovery
207,late_fees_added_after_tco_amount,late_fees_added_tco_recovery


In [14]:
old_names=rename['Source Field Names'].tolist()
target_name=rename['Target Field Name NEW(Display Name)'].tolist()

In [ ]:
for old_col, new_col in zip(old_names,target_name):
  target_rename_df=source_df.withColumnRenamed(old_col,new_col)

In [ ]:
target_rename_df.show()

In [ ]:
source_df.show()

In [ ]:
target_rename_df1=source_df.withColumnRenamed('customer_application_id', 'loan_customer_application_id')

In [ ]:
target_rename_df1.show()

In [15]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [16]:

mapping = dict(zip(old_names,target_name))
target=source_df.select([col(c).alias(mapping.get(c, c)) for c in source_df.columns])

In [17]:
target.show()

24/01/28 01:18:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----+----+--------------------+-------+----------------+-------+--------------------+-----------+----------------------------+----------------------+--------------------+---------------+-------------+-----------+--------------------+----------------+-----------------------+---------------------+-----------------------------+----------+--------------+-------------------------+------------------------------+---------+--------------+---------------------+---------------------+-------------+---------------+------------+-------------------------+------------------------------+----------------------------------+---------------------------------+---------------------------+-------------------------------+-------------------------+---------------------------+----------------------+-------------------+-----------------------+---------------------+-------------------------+---------------+---------------+---------------+-----------------+-----------------+-------------------------+--------

In [ ]:
target = target.withColumn(
    "age_of_book",
    when(
        (col("final_take_payment_date").isNotNull()) &
        (col("final_take_payment_date") <= col("measurement_date")),
        months_between(col("final_take_payment_date"), col("origination_date")).cast(IntegerType())
    ).when(
        (col("treasury_charge_off_date").isNotNull()) &
        (col("treasury_charge_off_date") <= col("measurement_date")),
        months_between(col("treasury_charge_off_date"), col("origination_date")).cast(IntegerType())
    ).when(
        col("origination_date") > col("measurement_date"),
        0
    ).otherwise(
        months_between(col("measurement_date"), col("origination_date")).cast(IntegerType())
    )
)

In [ ]:
window = Window.orderBy("measurement_date")
target = target.withColumn("temp_first_10_days_late", when(target.days_late_excl_charge_off >= 10, target.measurement_date))
target = target.withColumn("first_10_days_late_date", first(target.temp_first_10_days_late, True) \
        .over(window))

In [ ]:
target.select(col('first_10_days_late_date')).show(n=200)

In [ ]:
target = target.drop("temp_first_10_days_late")

In [ ]:
window = Window.orderBy(col("measurement_date").desc())

#target = target.withColumn("temp_first_60_days_late", when(col("days_late_excl_charge_off") >= 60, col("measurement_date")))
target = target.withColumn("temp_first_60_days_late", when(col("days_late_excl_charge_off") >= 60, col("measurement_date"))).filter(col("temp_first_60_days_late").isNotNull())
target = target.filter(col("temp_first_60_days_late").isNotNull())
target = target.withColumn("first_60_days_late_date", first("temp_first_60_days_late", True).over(window))

# Drop the temporary column if needed
target = target.drop("temp_first_60_days_late")

In [ ]:
target.select(col('temp_first_60_days_late'), col('days_late_excl_charge_off'), col('measurement_date')).show(n=300)

In [ ]:
target.select(col('first_60_days_late_date')).show()

In [ ]:
from pyspark.sql.functions import max as spark_max
target = target.withColumn(
    "current_maturity_date",
    when(
        col("status") == "paid_off",
        col("final_take_payment_date")
    ).when(
        (col("treasury_charge_off_date").isNotNull()) &
        (col("treasury_charge_off_date") <= col("measurement_date")),
        None
    ).when(
        col("settlement_flag") == "true",
        col("settlement_adjusted_end_date")
    ).otherwise(
        col("max_installment_date")
    )
)

In [ ]:
target.select(col('status')).show()

+-----------+
|     status|
+-----------+
|       late|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|       late|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
|charged_off|
+-----------+
only showing top 20 rows



In [ ]:
window = Window().orderBy("late_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Add the temporary column based on the condition
target = target.withColumn("temp_first_120_days_late", when(date_diff(target.late_date, target.measurement_date) >= 120, target.measurement_date))

# Add the final column using the window specification
target = target.withColumn("first_120_days_late_date", first("temp_first_120_days_late", True).over(window))

# Show the resulting DataFrame
target.show()

In [ ]:
target = target.withColumn("temp_most_recent_90_days_late_date",\
                           when(target.days_late_excl_charge_off == 90, target.measurement_date))
target = target.withColumn("most_recent_90_days_late_date", last(target.temp_most_recent_90_days_late_date, True) \
                           .over(Window.partitionBy(target.loan_id).orderBy(target.measurement_date)))

In [ ]:
target.select(col('most_recent_90_days_late_date')).show(n=200)

In [ ]:
window = Window().partitionBy("loan_id", "late_date")
target = target.withColumn("temp_number_of_instances_late", \
                           row_number() \
                           .over(Window.partitionBy(target.loan_id, target.late_date).orderBy(target.measurement_date)))
target = target.withColumn("number_of_instances_late", \
                           sum(when(
                               (target.prev_days_late_excl_charge_off.isNull() | (target.prev_days_late_excl_charge_off == 0)) \
                               & (target.temp_number_of_instances_late == 1) & (target.late_date.isNotNull()), 1).otherwise(0)) \
                           .over(window))

In [ ]:
target.select(col('number_of_instances_late')).show(n=200)

In [ ]:
target = target.withColumn("late_120_month",\
                           when(target.first_120_days_late_date.isNotNull(),\
                                trunc(target.first_120_days_late_date, 'month')).otherwise(None))

In [ ]:
target.select('late_120_month').show()